# Chat Completion Api

In [ ]:
%pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secret탭에  `OPENAI_API_KEY` 등록

In [8]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [9]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반가워요. 어떤 이야기를 나눌까요? 도움이 필요하시면 말씀해 주세요!'

## ⭐프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력 지시

## 기사 제목 교정
- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임펙트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 교정하세요.

# Steps
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

# Output Format
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

# Examples
- 원래 제목: '어제 서울에서 큰 불이 나 수백명이 대피했다.'
- 교정 제목: '서울 대형 화재, 수백명 대피'

- 원래 제목: '전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다.'
- 교정 제목: '전기차 판매량 급감에 웃는 내연차회사들'

# Extra Instructions
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사 제목을 간결히 수정해주세요.

{title_before}

"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 힘든 노동으로 끼니 거르는 노동자들의 고충


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
  # 역할/페르소나/지시사항
  system_message = """
  기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

  - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
  - 독자의 관심을 끌 수 있도록 간결하면서도 임펙트 있는 표현을 사용하세요.
  - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
  - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 교정하세요.

  # Steps
  1. 기사제목을 읽고 주요내용을 이해하세요.
  2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
  3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
  4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

  # Output Format
  기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

  - 원래 제목: [기사 원래 제목]
  - 교정 제목: [교정 기사 제목]

  # Examples
  - 원래 제목: '어제 서울에서 큰 불이 나 수백명이 대피했다.'
  - 교정 제목: '서울 대형 화재, 수백명 대피'

  - 원래 제목: '전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다.'
  - 교정 제목: '전기차 판매량 급감에 웃는 내연차회사들'

  # Extra Instructions
  - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
  - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
  - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

  """

  user_message = f"""
  다음 기사 제목을 간결히 수정해주세요.

  {title_before}

  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길
- 교정 제목: 주말 폭우 예상, 안전 유의하세요


## 연애코치 ReAct

In [4]:
# 함수 리팩토링

def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
  # 역할/페르소나/지시사항
  system_message = """
  << System Instruction >>
  어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

  << Output Format >>
  1. 상황분석:

  2. 행동계획:

  3. 실행:

  """

  user_message = f"""
  사용자 현재상황:
  {prompt}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [5]:
print(dating_coach('저 이번주말에 여자친구와 100일인데, 어떻하죠?'))

1. 상황분석:
   - 이번 주말에 여자친구와 100일 기념일을 맞이하는 상황입니다. 이는 두 분의 관계에 있어 중요한 이정표이므로, 특별한 기념일로 만들고 싶다는 마음이 클 것입니다. 현재 여자친구가 무엇을 기대할지, 또는 어떤 과거의 기념일에 기뻐했는지를 생각해보는 것이 중요합니다.

2. 행동계획:
   - 먼저 여자친구의 취향과 관심사를 고려하여 기념일에 맞는 특별한 계획을 세워보세요. 예를 들어, 이색적인 데이트 장소나 그녀가 좋아하는 음식을 만들어 주는 것 등을 고려할 수 있습니다.
   - 또한, 준비할 선물도 중요한 부분입니다. 소중한 추억이 담긴 사진첩이나 손편지 등 감정이 담긴 선물이나, 그녀의 취미에 맞는 아이템을 준비하는 것이 좋습니다.
   - 마지막으로, 특별한 날이기 때문에 여유 시간을 가지면서 서로의 감정과 소중한 순간을 공유하는 시간을 가져보세요.

3. 실행:
   - 먼저, 오늘 중으로 데이트 장소와 식사 준비를 결정합니다. 필요한 예약이나 준비물을 미리 체크해두세요.
   - 선물은 내일 중으로 완성하도록 하고, 만약 편지를 쓸 계획이라면 그 내용을 마음에서 우러나오는 대로 적어보세요.
   - 기념일 당일에는, 스케줄에 맞춰 우선적으로 여친을 위한 서프라이즈를 계획하고, 서로에게 감사의 마음을 전하며 잊지 못할 시간을 보내도록 노력하세요. 여친의 반응을 보며 대화를 이어가며 자연스럽게 추억을 나누는 것 또한 중요합니다. 

기념일이 두 분 사이의 사랑을 더욱 깊게 만들어주는 의미 있는 시간이 되길 바랍니다!


## 냉털 마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [6]:
# 함수 리팩토링

def fridge_raid_master(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
  # 역할/페르소나/지시사항
  system_message = """
  << System Instruction >>
  냉장고에 남아있는 음식재료를 알려주면, 해당 재료를 바탕으로 어떤 음식을 만들지를 조언해주세요.

  << Output Format >>
  1. 음식 이름:

  2. 추가 필요한 재료:

  3. 음식 레시피:

  """

  user_message = f"""
  냉장고에 남아있는 음식재료:
  {prompt}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [7]:
print(fridge_raid_master('오이, 두부, 대파, 계란'))

1. 음식 이름: 두부 오이 계란 샐러드

2. 추가 필요한 재료: 
   - 간장
   - 참기름
   - 식초 (선택사항)
   - 소금, 후추 (기본 양념)

3. 음식 레시피:
   1. 두부를 부드럽게 찌거나 데쳐서 먹기 좋은 크기로 자릅니다.
   2. 오이는 껍질을 벗기고 얇게 썰어줍니다.
   3. 대파는 잘게 썰어줍니다.
   4. 계란은 프라이팬에 계란 후라이로 만들어 줍니다.
   5. 큰 볼에 잘 자른 두부, 오이, 대파를 넣고 간장, 참기름, 소금, 후추로 간을 맞춥니다. 필요에 따라 식초도 추가하여 상큼함을 더할 수 있습니다.
   6. 마지막으로 볶은 계란을 얹고 잘 섞은 뒤, 그릇에 담아 맛있게 즐기세요!

맛있게 드세요!


### with 강사님

In [ ]:
def fridge_raid_master(user_foods, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 사용자 냉장고의 재료를 가지고 최고의 음식을 만들 수 있는 레시피를 추천하는 챗봇입니다.
현재 상황을 분석하고 실행계획을 세우며 추가 내용이 있을지 확인하는 꼼꼼함을 보여주세요.

## 지시사항

### 상황분석
1. **재료 확인**: 냉장고에 있는 재료 목록을 작성하고, 각 재료의 신선함과 사용 가능 시간을 파악합니다.
2. **요리 컨셉 설정**: 재료를 고려하여 요리의 주제를 정하고 잠재적인 요리 아이디어를 계획합니다.

### 실행계획
3. **목표 변경**: 사용 가능한 재료로 만들고자 하는 요리의 목표를 설정하고, 필요한 정보나 추가 자료를 조사합니다.
4. **전략 개발**: 단계별로 목표를 달성하기 위한 전반적인 요리 계획을 세웁니다.
    단, 사용자가 따라하기 쉽게 단계별 가이드를 작성해야 합니다.

### 검증 및 추가내용 확인
5. **레시피 검토**: 작성된 레시피를 검토하여 정보가 명확하고 완전한지 확인합니다.
6. **수정**: 필요 시 수정하여 최종 레시피를 완성합니다.

## 출력형식

1. 상황분석:
   - 재료 목록 및 상태
   - 요리 컨셉 및 아이디어

2. 실행계획:
   - 목표 및 세부 계획
   - 필요한 추가 자료 및 전략

3. 검증 및 추가내용 확인
   - 레시피의 정확성 검토
   - 수정 및 편집

4. 최종레시피
   - 사용자가 따라하기 쉽게 목록으로 작성
   - 필요한 준비물로 별도록 작성할것!

## Examples

- **Input**:
    사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
-   **Output**:
  1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
  2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
  3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

## Notes

- 요리의 문화적 관련성과 감수성을 고려합니다.
- 레시피는 예상 독자의 요리 지식 수준에 맞게 조정됩니다.

    """

    user_message = f"""
    사용자의 냉장고에는 현재 {user_foods}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [ ]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', '표고버섯', '명란젓', '된장']
display(Markdown(fridge_raid_master(user_foods)))

1. 상황분석:
   - 재료 목록 및 상태:
     - 소고기: 신선함이 좋음, 사용 가능 시간: 2-3일 남음
     - 양파: 신선함 양호, 사용 가능 시간: 1주일 남음
     - 표고버섯: 신선함 양호, 사용 가능 시간: 3-4일 남음
     - 명란젓: 유통기한 내에 있음, 사용 가능 시간: 2개월 이상
     - 된장: 유통기한 내에 있음, 사용 가능 시간: 1년 이상
  
   - 요리 컨셉 및 아이디어:
     오늘의 요리는 '국물 요리'를 주제로 하여, 소고기와 표고버섯으로 만든 된장찌개에 명란젓을 포함하여 풍미를 더해보겠습니다. 또한, 양파를 사용하여 맛을 깊게 할 계획입니다.

2. 실행계획:
   - 목표 및 세부 계획: 
     - 소고기와 표고버섯, 양파를 주재료로 한 된장찌개 만들기.
     - 명란젓을 곁들여 국물의 맛을 한층 업그레이드.
  
   - 필요한 추가 자료 및 전략:
     - 된장찌개의 기본 레시피 및 조리법을 확인하고, 명란젓을 추가하는 방법을 구체적으로 정리합니다.

3. 검증 및 추가내용 확인:
   - 레시피의 정확성 검토: 기본된장찌개 레시피에 명란젓 추가하는 부분을 명확하게 적어야 합니다. 또한, 조리 시간을 확인해 쉽게 따라할 수 있도록 할 수 있습니다.

4. 최종레시피:
   - 준비물:
     - 소고기 200g
     - 양파 1개
     - 표고버섯 100g
     - 된장 2큰술
     - 명란젓 1개 (50g)
     - 물 4컵 (약 1리터)

   - 요리 단계:
     1. 양파는 채썰고, 표고버섯은 슬라이스로 자릅니다.
     2. 소고기는 적당한 크기로 잘라 준비합니다.
     3. 냄비에 물을 붓고, 소고기를 넣어 끓입니다. (중불)
     4. 소고기가 익으면 양파와 표고버섯을 넣고 다시 끓입니다.
     5. 된장을 풀어 넣고, 원하는 간에 맞게 조절합니다. (중불에서 10분 정도 끓입니다)
     6. 마지막으로 명란젓을 넣고, 잠시 더 끓여서 완성합니다.
     7. 그릇에 담아 따뜻하게 즐기세요!

이 레시피는 쉽게 따라할 수 있으며, 필요한 재료들이 잘 활용됩니다. 만약 추가적으로 알고 싶은 정보가 있다면 말씀해 주세요!

## 면접 질문 생성 JSON 출력
- response_format={"type": "json_object"}

In [18]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
    매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.
    """

    user_message = f"""

    <<지시사항>>
    - 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
    - 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
    - 각 스킬별로 질문/답변을 3개씩 만들어주세요.

    <<출력형식>>
    출력은 json형식으로만 반환되어야 합니다.

    {{
        "hard_skill": [
            {{
                "question": "",
                "answer": "",
            }}
        ],
        "soft_skill": [
            {{
                "question": "",
                "answer": "",
            }}
        ]
    }}

    << 채용공고 >>
    {job_posting}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [19]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""

output = job_interview(job_posting)
print(type(output))
print(output)

<class 'str'>
{
    "hard_skill": [
        {
            "question": "AI 모델 배포를 위한 인프라를 구축할 때 가장 중요하게 고려해야 할 점은 무엇인가요?",
            "answer": "AI 모델 배포를 위한 인프라 구축 시 가장 중요하게 고려해야 할 점은 안정성과 확장성입니다. 안정성은 모델이 안정적으로 작동할 수 있도록 하며, 확장성은 사용량 증가에 대해 인프라를 쉽게 확장할 수 있도록 합니다. 또한, 모니터링과 로깅 시스템을 통해 성능을 지속적으로 분석하여 문제가 발생할 경우 신속하게 대응할 수 있도록 해야 합니다."
        },
        {
            "question": "컨테이너 오케스트레이션 도구인 Kubernetes를 사용할 때의 주요 장점은 무엇인가요?",
            "answer": "Kubernetes의 주요 장점은 서비스를 자동으로 배포, 스케일링 및 관리할 수 있는 기능입니다. 클러스터 내의 각 컨테이너의 상태를 모니터링하며, 필요에 따라 컨테이너의 수를 조절할 수 있습니다. 또한, 서비스 간의 로드밸런싱과 자원 관리 기능을 제공하여 시스템의 가용성과 효율성을 극대화할 수 있습니다."
        },
        {
            "question": "Jenkins를 사용한 CI/CD 파이프라인 구축 경험에 대해 말씀해 주세요.",
            "answer": "Jenkins를 활용하여 CI/CD 파이프라인을 구축한 경험이 있습니다. Git 저장소와 연동하여 코드 커밋 시 자동 빌드 및 테스트가 실행되도록 설정했습니다. 또한, 성공적인 빌드 후 자동 배포를 하는 프로세스를 설정하여 개발 주기를 단축시켰습니다. Jenkins의 플러그인을 활용하여 다양한 기능을 추가하고, 운영 환경에 맞는 태스크를 구체적으로 정의한 경험이 있습니다."
        }
    ],
    "soft_skill": [
     

In [20]:
import json

# 응답데이터가 json형식이 아니라면 오류 발생
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']

print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'AI 모델 배포를 위한 인프라를 구축할 때 가장 중요하게 고려해야 할 점은 무엇인가요?', 'answer': 'AI 모델 배포를 위한 인프라 구축 시 가장 중요하게 고려해야 할 점은 안정성과 확장성입니다. 안정성은 모델이 안정적으로 작동할 수 있도록 하며, 확장성은 사용량 증가에 대해 인프라를 쉽게 확장할 수 있도록 합니다. 또한, 모니터링과 로깅 시스템을 통해 성능을 지속적으로 분석하여 문제가 발생할 경우 신속하게 대응할 수 있도록 해야 합니다.'}, {'question': '컨테이너 오케스트레이션 도구인 Kubernetes를 사용할 때의 주요 장점은 무엇인가요?', 'answer': 'Kubernetes의 주요 장점은 서비스를 자동으로 배포, 스케일링 및 관리할 수 있는 기능입니다. 클러스터 내의 각 컨테이너의 상태를 모니터링하며, 필요에 따라 컨테이너의 수를 조절할 수 있습니다. 또한, 서비스 간의 로드밸런싱과 자원 관리 기능을 제공하여 시스템의 가용성과 효율성을 극대화할 수 있습니다.'}, {'question': 'Jenkins를 사용한 CI/CD 파이프라인 구축 경험에 대해 말씀해 주세요.', 'answer': 'Jenkins를 활용하여 CI/CD 파이프라인을 구축한 경험이 있습니다. Git 저장소와 연동하여 코드 커밋 시 자동 빌드 및 테스트가 실행되도록 설정했습니다. 또한, 성공적인 빌드 후 자동 배포를 하는 프로세스를 설정하여 개발 주기를 단축시켰습니다. Jenkins의 플러그인을 활용하여 다양한 기능을 추가하고, 운영 환경에 맞는 태스크를 구체적으로 정의한 경험이 있습니다.'}]
[{'question': '팀 내에서 기술 지원을 제공할 때 중요한 요소는 무엇이라고 생각하나요?', 'answer': '기술 지원을 제공할 때 가장 중요한 요소는 의사소통입니다. 알고리즘 개발자와의 원활한 커뮤니케이션을 통해 그들의 요구사항과 문제를 정확히 이해해야 합니다. 그에 맞춰 적절한 솔루션을 제시하고, 단